# combine the various datasets


In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import re
import os
os.chdir('/Users/simon/Desktop/uk-map/')
%matplotlib inline

In [78]:
# read in the various datasources

# conventional power sources based on DUKES table 5.11
# https://www.gov.uk/government/statistics/electricity-chapter-5-digest-of-united-kingdom-energy-statistics-dukes
# we have deliberately excluded renewables from this
# we will draw those in from other sources
# only includes "major power producers"
# only includes conventional sites >= 1MW

conv = pd.read_csv(
    'conventional-plant.csv', encoding='utf-8', lineterminator='\n')


In [79]:
# renewable sites based on REPD
# https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
# includes sites >= 1MW
# also includes storage; we will take those from RenewableUK data instead

re = pd.read_csv('renewable.csv', encoding='utf-8', lineterminator='\n')


In [80]:
# small-scale renewable sites based on Ofgem FiTs installation reports:
# https://www.ofgem.gov.uk/environmental-programmes/fit/contacts-guidance-and-resources/public-reports-and-data-fit/installation-reports
# contains anonymised data for all FiTs projects in the UK

# we have excluded FiTs projects ≥ 1MW as likely to appear in REPD
# this amounts to 1,500MW out of around 6,000MW

# we have then aggregated the data by fuel at local authority level
# aggregates of less than 1MW are excluded in the final data
# this amounts to 101MW of capacity in 2018, out of around 4,500MW total

# post code location data from here
# https://www.freemaptools.com/download-uk-postcode-lat-lng.htm

fits = pd.read_csv('fits.csv', encoding='utf-8', lineterminator='\n')


In [81]:
# storage sites other than pumped hydro
# data from RenewableUK
# data includes estimates of storage capacity (MWh)
# therefore also shows discharge times
# these are mostly 30-60 minutes for batteries
# we're not including that in the map data but it is available

# we don't know lifetime of batteries
# this paper suggests perhaps 10 years
# https://www.nrel.gov/docs/fy17osti/67102.pdf
# here we generously assume currently operating batteries remain open in 2030

storage = pd.read_csv('storage.csv', encoding='utf-8', lineterminator='\n')

storage = storage[storage.Type != 'Pumped Hydro']
storage = storage[[
    'Site', 'Company', 'Fuel', 'Type', 'Capacity (MW)', 'Region', 'Latitude',
    'Longitude', 'Low-Carbon', 'Renewable', 'CHP', 'Year Open', 'yearStart',
    'yearEnd'
]]


In [82]:
# interconnectors
# Ofgem list:
# https://www.ofgem.gov.uk/electricity/transmission-networks/electricity-interconnectors
# location data?
# http://ec.europa.eu/energy/infrastructure/transparency_platform/map-viewer/main.html

# good ref on status as of 2018
# https://www.ifri.org/sites/default/files/atoms/files/mathieu_deane_pye_brexit_2018.pdf
# includes 3 additional projects under consideration but not yet subject to regulatory review:
# Icelink 1,200MW
# GL-NL 1,000MW
# Nautilus 1,000-1,400MW

# our spreadsheet for found location data
# https://docs.google.com/spreadsheets/d/1Nf79pKZ5Qo7ZNdvH9eWQTuTp0i5LDSTPWgWoVF15y10/edit#gid=0


In [83]:
# CHP plants
# drawn from BEIS database
# https://chptools.decc.gov.uk/chp/public

# postcodes match with a database of lat/long
# https://github.com/dwyl/uk-postcodes-latitude-longitude-complete-csv

# excludes 99 sites <1MW, totalling 25MW of capacity
# excludes 6 sites without a postcode location match totalling 25MW
# excludes three sites already in the DUKES or REPD data

chp = pd.read_csv(
    'chp.csv', encoding='utf-8', lineterminator='\n')

In [84]:
# small gas and diesel plants
# drawn from the capacity market register
# postcodes extracted for each site by Aurora Energy Research for Carbon Brief

recip = pd.read_csv('recip.csv',encoding='utf-8', lineterminator ='\n')

In [85]:
# add some classifications to some of the data

re['Low-Carbon'] = 'Yes'
re['Renewable'] = 'Yes'
re['GWh (est)'] = np.nan

fits['Low-Carbon'] = 'Yes'
fits['Renewable'] = 'Yes'
fits['GWh (est)'] = np.nan
fits['Company'] = 'Small sites aggregated at local authority level '



In [89]:
# combine the sources

uk = conv.append(re) # conventional + REPD
uk = uk.append(fits) # add FiTs
uk = uk.append(storage) # add RUK battery storage
uk = uk.append(chp) # add CHP plants

# put the columns in a nice order

uk = uk[[
    'Site', 'Company', 'Fuel', 'Type', 'Capacity (MW)', 'Latitude',
    'Longitude', 'Low-Carbon', 'CHP', 'Year Open', 'yearStart',
    'yearEnd'
]]

In [87]:
# fix lines without fuel types

uk['Fuel'][uk.Site=='Pilsworth Landfill Site'] = 'Storage'
uk['Type'][uk.Site=='Pilsworth Landfill Site'] = 'Liquified air'

uk['Fuel'][uk.Site=='United Downs Project (Geothermal)'] = 'Geothermal'


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [88]:
# save the combined data

uk.to_csv('uk-map.csv',index=False,encoding='utf-8')

# Analysis and checks

In [26]:
uk[uk.Site.str.contains('Ironbridge')].sort_values(['Fuel','yearStart'])

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
142,Ironbridge,E.On UK,Coal,NaN,940.0,52.632529,-2.512676,No,NaN,1970.0,2011.0,2012


In [90]:
for year in range(2004, 2020):
    print str(year) + ': ' + str(uk['Capacity (MW)'][(uk.yearStart <= year) & (
        uk.yearEnd >= year)].count()) + ' sites (' + str(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Capacity (MW)'] >= 50)].count()) + ' ≥50MW; ' + str(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Capacity (MW)'] >= 100)].count()) + ' ≥100MW; ' + str(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Capacity (MW)'] >= 500)].count()) + ' ≥500MW)' + str(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Capacity (MW)'] >= 1000)].count()) + ' ≥1000MW) – mean ' + str(
                round(uk['Capacity (MW)'][(uk.yearStart <= year) & (
                    uk.yearEnd >= year)].mean())) + 'MW – median ' + str(
                        round(uk['Capacity (MW)'][(uk.yearStart <= year) & (
                            uk.yearEnd >= year)].median())) + 'MW'

2004: 483 sites (97 ≥50MW; 84 ≥100MW; 52 ≥500MW)29 ≥1000MW) – mean 156.0MW – median 4.0MW
2005: 533 sites (103 ≥50MW; 87 ≥100MW; 54 ≥500MW)30 ≥1000MW) – mean 149.0MW – median 4.0MW
2006: 577 sites (109 ≥50MW; 90 ≥100MW; 55 ≥500MW)31 ≥1000MW) – mean 142.0MW – median 4.0MW
2007: 609 sites (109 ≥50MW; 88 ≥100MW; 54 ≥500MW)30 ≥1000MW) – mean 133.0MW – median 4.0MW
2008: 789 sites (124 ≥50MW; 94 ≥100MW; 57 ≥500MW)32 ≥1000MW) – mean 110.0MW – median 5.0MW
2009: 843 sites (133 ≥50MW; 95 ≥100MW; 57 ≥500MW)33 ≥1000MW) – mean 104.0MW – median 5.0MW
2010: 902 sites (137 ≥50MW; 97 ≥100MW; 56 ≥500MW)32 ≥1000MW) – mean 98.0MW – median 5.0MW
2011: 1008 sites (147 ≥50MW; 103 ≥100MW; 60 ≥500MW)35 ≥1000MW) – mean 94.0MW – median 5.0MW
2012: 1117 sites (158 ≥50MW; 109 ≥100MW; 61 ≥500MW)35 ≥1000MW) – mean 88.0MW – median 5.0MW
2013: 1344 sites (162 ≥50MW; 107 ≥100MW; 58 ≥500MW)31 ≥1000MW) – mean 69.0MW – median 5.0MW
2014: 1647 sites (164 ≥50MW; 107 ≥100MW; 56 ≥500MW)31 ≥1000MW) – mean 56.0MW – median 5.0

In [91]:
print 'Renewable sites >1MW'
for year in range(2004, 2020):
    print str(year) + ': ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].count()
    ) + ' sites (' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 50) &
                            (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].count()
    ) + ' ≥50MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 100) &
                            (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].count()
    ) + ' ≥100MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 500) &
                            (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].count()
    ) + ' ≥500MW)' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 1000) &
                           (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].count()
    ) + ' ≥1000MW) – mean ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].mean())
    ) + 'MW – median ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                (uk['Low-Carbon'] == 'Yes') & (uk.Fuel != 'Nuclear')].median())) + 'MW'

Renewable sites >1MW
2004: 376 sites (12 ≥50MW; 5 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 14.0MW – median 3.0MW
2005: 422 sites (15 ≥50MW; 5 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 14.0MW – median 3.0MW
2006: 465 sites (20 ≥50MW; 7 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 15.0MW – median 3.0MW
2007: 501 sites (24 ≥50MW; 8 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 16.0MW – median 3.0MW
2008: 552 sites (30 ≥50MW; 9 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 16.0MW – median 3.0MW
2009: 600 sites (35 ≥50MW; 10 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 16.0MW – median 4.0MW
2010: 656 sites (38 ≥50MW; 12 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 17.0MW – median 4.0MW
2011: 755 sites (43 ≥50MW; 14 ≥100MW; 1 ≥500MW)1 ≥1000MW) – mean 16.0MW – median 4.0MW
2012: 863 sites (54 ≥50MW; 22 ≥100MW; 3 ≥500MW)1 ≥1000MW) – mean 18.0MW – median 4.0MW
2013: 1103 sites (68 ≥50MW; 29 ≥100MW; 6 ≥500MW)1 ≥1000MW) – mean 18.0MW – median 5.0MW
2014: 1409 sites (72 ≥50MW; 30 ≥100MW; 4 ≥500MW)2 ≥1000MW) – mean 16.0MW – median 5.0MW
2015: 1835 sites (77 ≥50M

In [92]:
print 'Fossil / nuclear'
for year in range(2004, 2020):
    print str(year) + ': ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].count()
    ) + ' sites (' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 50) &
                            ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].count()
    ) + ' ≥50MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 100) &
                            ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].count()
    ) + ' ≥100MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 500) &
                            ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].count()
    ) + ' ≥500MW)' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 1000) &
                           ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].count()
    ) + ' ≥1000MW) – mean ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].mean())
    ) + 'MW – median ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                ((uk['Low-Carbon'] == 'No') | (uk.Fuel == 'Nuclear'))].median())) + 'MW'

Fossil / nuclear
2004: 107 sites (85 ≥50MW; 79 ≥100MW; 51 ≥500MW)28 ≥1000MW) – mean 652.0MW – median 434.0MW
2005: 111 sites (88 ≥50MW; 82 ≥100MW; 53 ≥500MW)29 ≥1000MW) – mean 663.0MW – median 434.0MW
2006: 112 sites (89 ≥50MW; 83 ≥100MW; 54 ≥500MW)30 ≥1000MW) – mean 668.0MW – median 442.0MW
2007: 108 sites (85 ≥50MW; 80 ≥100MW; 53 ≥500MW)29 ≥1000MW) – mean 677.0MW – median 427.0MW
2008: 237 sites (94 ≥50MW; 85 ≥100MW; 56 ≥500MW)31 ≥1000MW) – mean 329.0MW – median 15.0MW
2009: 243 sites (98 ≥50MW; 85 ≥100MW; 56 ≥500MW)32 ≥1000MW) – mean 321.0MW – median 15.0MW
2010: 246 sites (99 ≥50MW; 85 ≥100MW; 55 ≥500MW)31 ≥1000MW) – mean 314.0MW – median 16.0MW
2011: 253 sites (104 ≥50MW; 89 ≥100MW; 59 ≥500MW)34 ≥1000MW) – mean 326.0MW – median 17.0MW
2012: 254 sites (104 ≥50MW; 87 ≥100MW; 58 ≥500MW)34 ≥1000MW) – mean 325.0MW – median 18.0MW
2013: 241 sites (94 ≥50MW; 78 ≥100MW; 52 ≥500MW)30 ≥1000MW) – mean 298.0MW – median 15.0MW
2014: 238 sites (92 ≥50MW; 77 ≥100MW; 52 ≥500MW)29 ≥1000MW) – mean 

In [93]:
print 'Coal'
for year in range(2004, 2020):
    print str(year) + ': ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].count()
    ) + ' sites (' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 50) &
                           ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].count()
    ) + ' ≥50MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 100) &
                            ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].count()
    ) + ' ≥100MW; ' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 500) &
                            ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].count()
    ) + ' ≥500MW)' + str(
        uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                            (uk['Capacity (MW)'] >= 1000) &
                           ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].count()
    ) + ' ≥1000MW) – mean ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                                ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].mean())
    ) + 'MW – median ' + str(
        round(
            uk['Capacity (MW)'][(uk.yearStart <= year) & (uk.yearEnd >= year) &
                               ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))].median())) + 'MW'

Coal
2004: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1604.0MW – median 1933.0MW
2005: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1605.0MW – median 1940.0MW
2006: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1605.0MW – median 1940.0MW
2007: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1607.0MW – median 1948.0MW
2008: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1614.0MW – median 1948.0MW
2009: 18 sites (18 ≥50MW; 18 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1616.0MW – median 1949.0MW
2010: 19 sites (19 ≥50MW; 19 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1546.0MW – median 1940.0MW
2011: 20 sites (20 ≥50MW; 20 ≥100MW; 17 ≥500MW)15 ≥1000MW) – mean 1565.0MW – median 1949.0MW
2012: 19 sites (19 ≥50MW; 19 ≥100MW; 16 ≥500MW)14 ≥1000MW) – mean 1592.0MW – median 1958.0MW
2013: 14 sites (14 ≥50MW; 14 ≥100MW; 11 ≥500MW)10 ≥1000MW) – mean 1534.0MW – median 1960.0MW
2014: 13 sites (13 ≥50MW; 13 ≥100MW; 11 ≥500MW)9 ≥1000MW) – mean 

In [76]:
uk[uk.Fuel=='Coal']

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1455.0,51.387006,-3.406215,No,NaN,1971.0,2004.0,2007
2,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008.0,2017
3,Aberthaw B,RWE Generation SE,Coal,NaN,1610.0,51.387006,-3.406215,No,NaN,1971.0,2018.0,2018
49,Cockenzie,Scottish Power,Coal,NaN,1152.0,55.968323,-2.972481,No,NaN,1967.0,2004.0,2012
61,Cottam,EDF Energy,Coal,NaN,2008.0,53.304764,-0.780451,No,NaN,1969.0,2004.0,2018
76,Didcot A,RWE Innogy Plc,Coal,NaN,1925.0,51.622252,-1.260853,No,NaN,1972.0,2004.0,2004
77,Didcot A,RWE Npower Plc,Coal,NaN,1940.0,51.622252,-1.260853,No,NaN,1972.0,2005.0,2006
78,Didcot A,RWE Npower Plc,Coal,NaN,1958.0,51.622252,-1.260853,No,NaN,1972.0,2007.0,2012
85,Drax,Drax Power Ltd,Coal,NaN,3870.0,53.737546,-0.999021,No,NaN,1974.0,2004.0,2012
91,Drax coal,Drax Power Ltd,Coal,NaN,3225.0,53.737546,-0.999021,No,NaN,1974.0,2013.0,2013


In [48]:
uk[(uk.yearStart <= 2008) & (uk.yearEnd >= 2008) &
                            ((uk['Low-Carbon'] == 'No') & (uk.Fuel == 'Coal'))]

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
1,Aberthaw B,RWE Npower Plc,Coal,NaN,1586.0,51.387006,-3.406215,No,NaN,1971.0,2008.0,2017
48,Cottam,EDF Energy,Coal,NaN,2008.0,53.304764,-0.780451,No,NaN,1969.0,2004.0,2018
68,Drax,Drax Power Ltd,Coal,NaN,3870.0,53.737546,-0.999021,No,NaN,1974.0,2004.0,2012
82,Eggborough,Eggborough Power Ltd,Coal,NaN,1960.0,53.712671,-1.124850,No,NaN,1967.0,2004.0,2018
92,Ferrybridge C,Scottish & Southern Energy plc,Coal,NaN,1955.0,53.715638,-1.280574,No,NaN,1966.0,2004.0,2008
96,Fiddler’s Ferry,SSE,Coal,NaN,1961.0,53.371950,-2.686922,No,NaN,1971.0,2004.0,2018
162,Kilroot,AES,Coal,NaN,560.0,54.725333,-5.766401,No,NaN,1981.0,2004.0,2018
166,Kingsnorth,E.On UK,Coal,NaN,1940.0,51.416713,0.603039,No,NaN,1970.0,2004.0,2012
188,Longannet,Scottish Power,Coal,NaN,2304.0,56.048549,-3.682552,No,NaN,1970.0,2004.0,2013
190,Lynemouth,Lynemouth Power Ltd,Coal,NaN,420.0,55.204750,-1.520356,No,NaN,1972.0,2004.0,2015


In [12]:
uk[uk.Longitude < -10]

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd


In [27]:
uk[uk.Fuel.isnull()]

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd


In [ ]:
# some bits of random analysis

In [13]:
uk[uk.Site=='Cornwall'].head()

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd


In [26]:
uk[uk.Type=='Battery'].pivot_table(index='yearStart',values='Capacity (MW)',aggfunc='sum')

,Capacity (MW)
yearStart,
2006,0.005
2008,0.076
2009,0.012
2013,2.295
2014,5.750
2015,0.295
2016,10.595
2017,49.000
2018,246.000


In [27]:
uk[uk.Type=='Battery'].sort_values(by='Capacity (MW)',ascending=False)

,Site,Company,Fuel,Type,Capacity (MW),Region,Latitude,Longitude,Low-Carbon,Renewable,CHP,Year Open,yearStart,yearEnd
2569,Huggin's Hall,Low Carbon,Storage,Battery,50.000,South East,51.093574,0.508258,Yes,Yes,n/a,2018.0,2018,2019
2565,West Burton,EDF Energy Renewables,Storage,Battery,49.000,East Midlands,53.364529,-0.798356,Yes,Yes,n/a,2018.0,2018,2019
2563,Roosecote,Centrica,Storage,Battery,49.000,North West,54.105223,-3.190031,Yes,Yes,n/a,2018.0,2018,2019
2562,Pelham,Statera Energy/ InfraRed Capital,Storage,Battery,49.000,Eastern,51.937437,0.119184,Yes,Yes,n/a,2017.0,2017,2019
2566,Tynemouth Energy Storage System (TESS),Enel (formerly Element Power),Storage,Battery,25.000,North East,55.025976,-1.503105,Yes,Yes,n/a,2018.0,2018,2019
2568,Pen y Cymoedd Energy Storage,Vattenfall,Storage,Battery,22.000,Wales,51.655543,-3.695730,Yes,Yes,n/a,2018.0,2018,2019
2570,Land at Carnegie Road,Ørsted,Storage,Battery,20.000,North West,53.419931,-2.927637,Yes,Yes,n/a,2018.0,2018,2019
2542,AES Kilroot Station Battery Storage Array,AES Kilroot Power (AES KPL),Storage,Battery,10.000,Northern Ireland,54.724407,-5.767212,Yes,Yes,n/a,2016.0,2016,2019
2656,Blackburn Meadows,E.On,Storage,Battery,10.000,Yorkshire and Humber,53.420135,-1.398460,Yes,Yes,n/a,NaN,2019,2019
2571,Cleator,Low Carbon Storage Investment Company/ VLC,Storage,Battery,10.000,North West,54.503359,-3.529301,Yes,Yes,n/a,2018.0,2018,2019


In [49]:
uk['Capacity (MW)'][(uk.Fuel=='Wind')&(uk.yearEnd==2018)].sum()

1117.751

In [54]:
uk[uk.Site.str.contains('sund')]

,Site,Company,Fuel,Type,Capacity (MW),Latitude,Longitude,Low-Carbon,CHP,Year Open,yearStart,yearEnd
13,Iggesund Paperboard CHP Plant,Iggesund Paperboard (Workington),Biomass,NaN,49.0,54.663976,-3.547331,Yes,Yes,2013.0,2013.0,2019
50,Iggesund Paperboard (Workington) Ltd,NaN,Biomass,NaN,49.9,NaN,NaN,Yes,Yes,NaN,2008.0,2019
